In [4]:
# === BƯỚC 1: KẾT NỐI GOOGLE DRIVE (để lưu file) ===
# from google.colab import drive
# drive.mount('/content/drive')

# Tạo thư mục lưu dữ liệu
import os
output_dir = '/content/Bus_Data_Simulation'
os.makedirs(output_dir, exist_ok=True)
print(f"Dữ liệu sẽ được lưu tại: {output_dir}")

Dữ liệu sẽ được lưu tại: /content/Bus_Data_Simulation


In [5]:
# === BƯỚC 2: TẠO DỮ LIỆU GIẢ LẬP ===
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta

# --- 1. Thời gian: 6h00 → 18h00, mỗi 15 phút ---
start_time = datetime(2025, 9, 4, 6, 0, 0)
end_time = datetime(2025, 9, 4, 18, 0, 0)
time_slots = []
current = start_time
while current <= end_time:
    time_slots.append(current)
    current += timedelta(minutes=15)
print(f"Đã tạo {len(time_slots)} khung thời gian (15 phút/lần)")

# --- 2. 5 tuyến xe buýt, mỗi tuyến 5 trạm ---
bus_routes = {
    'Tuyen_A': ['A1_BenThanh', 'A2_LeLoi', 'A3_NguyenHue', 'A4_ThuKhoaHuan', 'A5_Quan7'],
    'Tuyen_B': ['B1_Cholon', 'B2_AnDuongVuong', 'B3_HauGiang', 'B4_KinhDuongVuong', 'B5_BinhTan'],
    'Tuyen_C': ['C1_TanSonNhat', 'C2_TruongSon', 'C3_CongHoa', 'C4_LyThuongKiet', 'C5_QuanTanBinh'],
    'Tuyen_D': ['D1_ThuDuc', 'D2_VoVanNgan', 'D3_DamHuu', 'D4_XaLoHaNoi', 'D5_ThuDucUni'],
    'Tuyen_E': ['E1_BinhThanh', 'E2_DienBienPhu', 'E3_NguyenThiMinhKhai', 'E4_Pasteur', 'E5_Quan3']
}

# Tọa độ giả lập (gần TP.HCM)
station_coords = {}
lat_base, lon_base = 10.75, 106.65
np.random.seed(42)  # Để kết quả ổn định
for route, stations in bus_routes.items():
    for i, station in enumerate(stations):
        lat = lat_base + 0.015 * i + np.random.uniform(-0.005, 0.005)
        lon = lon_base + 0.025 * i + np.random.uniform(-0.005, 0.005)
        station_coords[station] = (round(lat, 6), round(lon, 6))

# --- 3. Hàm tạo nhu cầu hành khách ---
def generate_passenger_demand():
    data = []
    for route, stations in bus_routes.items():
        for station in stations:
            for t in time_slots:
                hour = t.hour
                # Cao điểm: 7-9h & 16-18h
                if hour in [7, 8, 16, 17]:
                    mean = 120
                    std = 30
                elif hour in [6, 9, 15, 18]:
                    mean = 80
                    std = 20
                else:
                    mean = 40
                    std = 15
                passengers = max(0, int(np.random.normal(mean, std)))
                data.append({
                    'route': route,
                    'station': station,
                    'time': t.strftime("%Y-%m-%d %H:%M:%S"),
                    'passengers': passengers
                })
    return pd.DataFrame(data)

# --- 4. Thời tiết ---
def generate_weather():
    data = []
    for t in time_slots:
        temp = np.random.uniform(26, 34)
        rain = np.random.choice([0, 0.5, 1, 2], p=[0.7, 0.15, 0.1, 0.05])
        data.append({
            'time': t.strftime("%Y-%m-%d %H:%M:%S"),
            'temperature': round(temp, 1),
            'rain_mm': rain
        })
    return pd.DataFrame(data)

# --- 5. Sự kiện ---
def generate_events():
    data = []
    for t in time_slots:
        hour = t.hour
        event = 'none'
        factor = 1.0
        if hour in [7, 8, 16, 17] and random.random() < 0.25:
            event = random.choice(['traffic_jam', 'festival'])
            factor = 1.5 if event == 'traffic_jam' else 1.7
        data.append({
            'time': t.strftime("%Y-%m-%d %H:%M:%S"),
            'event': event,
            'demand_factor': factor
        })
    return pd.DataFrame(data)

# --- 6. GPS xe buýt (3 xe/tuyến) ---
def generate_gps():
    data = []
    for route in bus_routes.keys():
        for bus_id in range(1, 4):
            for t in time_slots:
                station = random.choice(bus_routes[route])
                lat, lon = station_coords[station]
                data.append({
                    'route': route,
                    'bus_id': f"{route}_Bus_{bus_id}",
                    'time': t.strftime("%Y-%m-%d %H:%M:%S"),
                    'latitude': round(lat + np.random.uniform(-0.003, 0.003), 6),
                    'longitude': round(lon + np.random.uniform(-0.003, 0.003), 6)
                })
    return pd.DataFrame(data)

# --- 7. Lộ trình trạm ---
def generate_stops():
    data = []
    for route, stations in bus_routes.items():
        for i, station in enumerate(stations):
            lat, lon = station_coords[station]
            data.append({
                'route': route,
                'station': station,
                'order': i + 1,
                'latitude': lat,
                'longitude': lon
            })
    return pd.DataFrame(data)

# === CHẠY TẤT CẢ ===
print("Đang tạo dữ liệu...")

demand_df = generate_passenger_demand()
weather_df = generate_weather()
event_df = generate_events()
gps_df = generate_gps()
stops_df = generate_stops()

# Gộp dữ liệu
merged = demand_df.merge(weather_df, on='time').merge(event_df, on='time')
merged['adjusted_demand'] = (merged['passengers'] * merged['demand_factor']).astype(int)

# === LƯU FILE VÀO GOOGLE DRIVE ===
merged.to_csv(f'{output_dir}/bus_demand_integrated.csv', index=False)
stops_df.to_csv(f'{output_dir}/bus_stops.csv', index=False)
gps_df.to_csv(f'{output_dir}/bus_gps.csv', index=False)

print("HOÀN TẤT! Dữ liệu đã được lưu:")
print(f"   - bus_demand_integrated.csv → {len(merged):,} bản ghi")
print(f"   - bus_stops.csv → {len(stops_df)} trạm")
print(f"   - bus_gps.csv → {len(gps_df):,} vị trí GPS")

# === XEM MẪU DỮ LIỆU ===
from IPython.display import display
print("\nMẪU DỮ LIỆU NHU CẦU HÀNH KHÁCH:")
display(merged[['route', 'station', 'time', 'passengers', 'event', 'adjusted_demand']].head(10))

print("\nMẪU LỘ TRÌNH TRẠM:")
display(stops_df.head(10))

Đã tạo 49 khung thời gian (15 phút/lần)
Đang tạo dữ liệu...
HOÀN TẤT! Dữ liệu đã được lưu:
   - bus_demand_integrated.csv → 1,225 bản ghi
   - bus_stops.csv → 25 trạm
   - bus_gps.csv → 735 vị trí GPS

MẪU DỮ LIỆU NHU CẦU HÀNH KHÁCH:


,route,station,time,passengers,event,adjusted_demand
0,Tuyen_A,A1_BenThanh,2025-09-04 06:00:00,94,none,94
1,Tuyen_A,A1_BenThanh,2025-09-04 06:15:00,83,none,83
2,Tuyen_A,A1_BenThanh,2025-09-04 06:30:00,77,none,77
3,Tuyen_A,A1_BenThanh,2025-09-04 06:45:00,73,none,73
4,Tuyen_A,A1_BenThanh,2025-09-04 07:00:00,75,none,75
5,Tuyen_A,A1_BenThanh,2025-09-04 07:15:00,98,none,98
6,Tuyen_A,A1_BenThanh,2025-09-04 07:30:00,106,traffic_jam,159
7,Tuyen_A,A1_BenThanh,2025-09-04 07:45:00,151,none,151
8,Tuyen_A,A1_BenThanh,2025-09-04 08:00:00,130,traffic_jam,195
9,Tuyen_A,A1_BenThanh,2025-09-04 08:15:00,67,traffic_jam,100



MẪU LỘ TRÌNH TRẠM:


,route,station,order,latitude,longitude
0,Tuyen_A,A1_BenThanh,1,10.748745,106.654507
1,Tuyen_A,A2_LeLoi,2,10.767320,106.675987
2,Tuyen_A,A3_NguyenHue,3,10.776560,106.696560
3,Tuyen_A,A4_ThuKhoaHuan,4,10.790581,106.728662
4,Tuyen_A,A5_Quan7,5,10.811011,106.752081
5,Tuyen_B,B1_Cholon,1,10.745206,106.654699
6,Tuyen_B,B2_AnDuongVuong,2,10.768324,106.672123
7,Tuyen_B,B3_HauGiang,3,10.776818,106.696834
8,Tuyen_B,B4_KinhDuongVuong,4,10.793042,106.725248
9,Tuyen_B,B5_BinhTan,5,10.809319,106.747912


In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

import os
data_dir = '/content/Bus_Data_Simulation'
print(f"Dữ liệu nằm tại: {data_dir}")

Dữ liệu nằm tại: /content/Bus_Data_Simulation


In [9]:
# Cài thư viện
!pip install tensorflow scikit-learn matplotlib -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [10]:
# Đọc dữ liệu đã tạo
df = pd.read_csv(f'{data_dir}/bus_demand_integrated.csv')
stops_df = pd.read_csv(f'{data_dir}/bus_stops.csv')

print(f"Đã tải {len(df):,} bản ghi nhu cầu hành khách")
print(f"Các tuyến: {df['route'].unique()}")

Đã tải 1,225 bản ghi nhu cầu hành khách
Các tuyến: ['Tuyen_A' 'Tuyen_B' 'Tuyen_C' 'Tuyen_D' 'Tuyen_E']


In [11]:
# Chuyển time thành datetime
df['time'] = pd.to_datetime(df['time'])

# Tạo danh sách thời gian cho ngày tiếp theo (5/9/2025)
future_start = datetime(2025, 9, 5, 6, 0, 0)
future_times = [future_start + timedelta(minutes=15*i) for i in range(48)]  # 48 khung

# Hàm tạo chuỗi thời gian
def create_dataset(data, time_steps=8):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

# Chuẩn bị scaler
scaler = MinMaxScaler()

In [12]:
# Lưu kết quả dự đoán
all_predictions = []

# Huấn luyện riêng cho từng tuyến
for route in df['route'].unique():
    print(f"\nĐang huấn luyện LSTM cho tuyến: {route}")

    # Lọc dữ liệu tuyến
    route_df = df[df['route'] == route].copy()
    route_df = route_df.sort_values('time')

    # Gom nhu cầu theo thời gian (tổng tất cả trạm)
    demand_by_time = route_df.groupby('time')['adjusted_demand'].sum().reset_index()

    # Chuẩn hóa
    values = demand_by_time['adjusted_demand'].values.reshape(-1, 1)
    scaled = scaler.fit_transform(values)

    # Tạo dữ liệu huấn luyện
    time_steps = 8  # Dùng 8 khung trước để dự đoán
    X, y = create_dataset(scaled, time_steps)
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Xây dựng mô hình LSTM
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(time_steps, 1)),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')

    # Huấn luyện
    model.fit(X, y, epochs=30, batch_size=16, verbose=0)

    # Dự đoán cho ngày 5/9
    last_sequence = scaled[-time_steps:].reshape(1, time_steps, 1)
    future_pred = []

    for _ in range(48):  # 48 khung 15 phút
        pred = model.predict(last_sequence, verbose=0)
        future_pred.append(pred[0, 0])
        last_sequence = np.append(last_sequence[:, 1:, :], pred.reshape(1, 1, 1), axis=1)

    # Đảo ngược chuẩn hóa
    future_pred = scaler.inverse_transform(np.array(future_pred).reshape(-1, 1)).flatten()

    # Lưu kết quả
    pred_df = pd.DataFrame({
        'route': route,
        'time': future_times,
        'predicted_demand': future_pred.astype(int)
    })
    all_predictions.append(pred_df)

    print(f"Hoàn thành dự đoán {route}: trung bình {future_pred.mean():.0f} hành khách/giờ")


Đang huấn luyện LSTM cho tuyến: Tuyen_A
Hoàn thành dự đoán Tuyen_A: trung bình 504 hành khách/giờ

Đang huấn luyện LSTM cho tuyến: Tuyen_B
Hoàn thành dự đoán Tuyen_B: trung bình 406 hành khách/giờ

Đang huấn luyện LSTM cho tuyến: Tuyen_C
Hoàn thành dự đoán Tuyen_C: trung bình 418 hành khách/giờ

Đang huấn luyện LSTM cho tuyến: Tuyen_D
Hoàn thành dự đoán Tuyen_D: trung bình 552 hành khách/giờ

Đang huấn luyện LSTM cho tuyến: Tuyen_E
Hoàn thành dự đoán Tuyen_E: trung bình 521 hành khách/giờ


In [13]:
# Gộp tất cả dự đoán
final_pred = pd.concat(all_predictions)
final_pred['time_str'] = final_pred['time'].dt.strftime('%H:%M')

# Lưu file
output_file = f'{data_dir}/bus_demand_prediction_20250905.csv'
final_pred[['route', 'time', 'predicted_demand']].to_csv(output_file, index=False)
print(f"\nĐÃ LƯU DỰ ĐOÁN NGÀY 5/9 TẠI:\n{output_file}")


ĐÃ LƯU DỰ ĐOÁN NGÀY 5/9 TẠI:
/content/Bus_Data_Simulation/bus_demand_prediction_20250905.csv


In [14]:
# Vẽ biểu đồ tổng nhu cầu theo giờ
plt.figure(figsize=(14, 8))
for route in final_pred['route'].unique():
    route_data = final_pred[final_pred['route'] == route]
    hourly = route_data.groupby(route_data['time'].dt.hour)['predicted_demand'].sum()
    plt.plot(hourly.index, hourly.values, marker='o', label=route)

plt.title('DỰ ĐOÁN TỔNG NHU CẦU HÀNH KHÁCH THEO GIỜ - NGÀY 5/9/2025', fontsize=14)
plt.xlabel('Giờ trong ngày')
plt.ylabel('Tổng hành khách dự đoán')
plt.xticks(range(6, 19))
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [16]:
# from google.colab import drive
# drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Đường dẫn
DATA_DIR = '/content/Bus_Data_Simulation'
pred_file = f'{DATA_DIR}/bus_demand_prediction_20250905.csv'
stops_file = f'{DATA_DIR}/bus_stops.csv'

# Đọc dữ liệu
pred_df = pd.read_csv(pred_file)
stops_df = pd.read_csv(stops_file)
pred_df['time'] = pd.to_datetime(pred_df['time'])

print(f"Đã tải dự đoán LSTM: {len(pred_df)} bản ghi")
print(f"Các tuyến: {pred_df['route'].unique().tolist()}")

Đã tải dự đoán LSTM: 240 bản ghi
Các tuyến: ['Tuyen_A', 'Tuyen_B', 'Tuyen_C', 'Tuyen_D', 'Tuyen_E']


In [17]:
# === THAM SỐ HỆ THỐNG ===
BUS_CAPACITY = 50        # Số khách tối đa/xe
MIN_WAIT = 5             # Tối thiểu 5 phút/chuyến
MAX_WAIT = 20            # Tối đa 20 phút/chuyến
NUM_ANTS = 30            # Số kiến
ITERATIONS = 100         # Số vòng lặp
EVAPORATION = 0.5        # Tốc độ bay hơi pheromone
ALPHA = 1.0              # Tầm quan trọng pheromone
BETA = 2.0               # Tầm quan trọng heuristic

In [20]:
def run_aco_for_route(route_name, demand_df, time_slots):
    # Lọc dữ liệu
    route_demand = demand_df[demand_df['route'] == route_name].copy()
    route_demand['time_slot'] = route_demand['time'].dt.floor('15min')
    demand_by_slot = route_demand.groupby('time_slot')['predicted_demand'].sum()

    # Tạo vector nhu cầu theo time_slots
    demand_values = np.zeros(len(time_slots))
    for i, t in enumerate(time_slots):
        if t in demand_by_slot.index:
            demand_values[i] = demand_by_slot[t]

    if demand_values.sum() == 0:
        return [], 0

    # Pheromone
    pheromone = np.ones(len(time_slots)) * 1.0
    best_schedule = []
    best_score = -float('inf')

    for _ in range(ITERATIONS):
        for ant in range(NUM_ANTS):
            schedule = []
            current_time = 0
            remaining_capacity = BUS_CAPACITY

            while current_time < len(time_slots):
                # Xác suất khởi hành
                if remaining_capacity <= 0:
                    break

                prob = (pheromone[current_time] ** ALPHA) * ((demand_values[current_time] / (demand_values[current_time] + 1)) ** BETA)
                if random.random() < prob and (not schedule or (current_time - schedule[-1]) >= 1):  # cách ít nhất 15 phút
                    schedule.append(current_time)
                    remaining_capacity = BUS_CAPACITY  # xe mới
                current_time += 1

            if not schedule:
                continue

            # Tính fitness
            total_wait = 0
            total_served = 0
            trips = len(schedule)

            for start_idx in schedule:
                served_this_trip = 0
                for t_idx in range(start_idx, len(time_slots)):
                    wait = (t_idx - start_idx) * 15
                    if wait > MAX_WAIT:
                        break
                    can_serve = min(demand_values[t_idx], BUS_CAPACITY - served_this_trip)
                    if can_serve > 0:
                        total_wait += can_serve * wait
                        total_served += can_serve
                        served_this_trip += can_serve
                        if served_this_trip >= BUS_CAPACITY:
                            break

            if total_served == 0:
                continue

            avg_wait = total_wait / total_served
            load_factor = total_served / (trips * BUS_CAPACITY)
            score = load_factor * 100 - avg_wait

            if score > best_score:
                best_score = score
                best_schedule = schedule.copy()

        # Cập nhật pheromone
        pheromone *= (1 - EVAPORATION)
        if best_schedule:
            for idx in best_schedule:
                pheromone[idx] += best_score / 10

    return best_schedule, best_score

In [21]:
# Đảm bảo các biến cần thiết
BUS_CAPACITY = 50
MAX_WAIT = 30
NUM_ANTS = 30
ITERATIONS = 80
ALPHA = 1.0
BETA = 3.0
EVAPORATION = 0.5

# Tạo time_slots
time_slots = sorted(pred_df['time'].dt.floor('15min').unique())

# Chạy ACO
final_schedule = []
print("ĐANG CHẠY ACO CHO 5 TUYẾN...")

for route in pred_df['route'].unique():
    print(f"   → Đang tối ưu: {route}", end="")
    schedule_idx, score = run_aco_for_route(route, pred_df, time_slots)

    if not schedule_idx:
        print(" → Không có chuyến")
        continue

    print(f" → {len(schedule_idx)} chuyến, score: {score:.1f}")

    for start_idx in schedule_idx:
        start_time = time_slots[start_idx]
        served = 0
        for t_idx in range(start_idx, len(time_slots)):
            wait = (t_idx - start_idx) * 15
            if wait > MAX_WAIT:
                break
            demand = pred_df[
                (pred_df['route'] == route) &
                (pred_df['time'].dt.floor('15min') == time_slots[t_idx])
            ]['predicted_demand'].sum()
            add = min(demand, BUS_CAPACITY - served)
            if add <= 0:
                break
            served += add

        final_schedule.append({
            'route': route,
            'departure_time': start_time,
            'served_passengers': min(served, BUS_CAPACITY),
            'load_factor': f"{min(served, BUS_CAPACITY)/BUS_CAPACITY:.1%}"
        })

# Tạo DataFrame (AN TOÀN)
if final_schedule:
    schedule_df = pd.DataFrame(final_schedule)
    schedule_df = schedule_df.sort_values(['route', 'departure_time'])
    output_aco = f'{DATA_DIR}/bus_optimal_schedule_20250905.csv'
    schedule_df.to_csv(output_aco, index=False)
    print(f"\nHOÀN TẤT! ĐÃ TẠO {len(schedule_df)} CHUYẾN!")
    print(f"Lịch trình lưu tại: {output_aco}")
else:
    print("\nKHÔNG TẠO ĐƯỢC LỊCH TRÌNH NÀO!")
    schedule_df = pd.DataFrame(columns=['route', 'departure_time', 'served_passengers', 'load_factor'])

ĐANG CHẠY ACO CHO 5 TUYẾN...
   → Đang tối ưu: Tuyen_A → 48 chuyến, score: 100.0
   → Đang tối ưu: Tuyen_B → 48 chuyến, score: 100.0
   → Đang tối ưu: Tuyen_C → 48 chuyến, score: 100.0
   → Đang tối ưu: Tuyen_D → 48 chuyến, score: 100.0
   → Đang tối ưu: Tuyen_E → 48 chuyến, score: 100.0

HOÀN TẤT! ĐÃ TẠO 240 CHUYẾN!
Lịch trình lưu tại: /content/Bus_Data_Simulation/bus_optimal_schedule_20250905.csv


In [22]:
from IPython.display import display
if len(schedule_df) > 0:
    print("10 CHUYẾN ĐẦU TIÊN:")
    display(schedule_df.head(10))

    print("\nTỔNG HỢP:")
    summary = schedule_df.groupby('route').size().reset_index(name='Số chuyến')
    display(summary)
else:
    print("Không có dữ liệu để hiển thị.")

10 CHUYẾN ĐẦU TIÊN:


,route,departure_time,served_passengers,load_factor
0,Tuyen_A,2025-09-05 06:00:00,50,100.0%
1,Tuyen_A,2025-09-05 06:15:00,50,100.0%
2,Tuyen_A,2025-09-05 06:30:00,50,100.0%
3,Tuyen_A,2025-09-05 06:45:00,50,100.0%
4,Tuyen_A,2025-09-05 07:00:00,50,100.0%
5,Tuyen_A,2025-09-05 07:15:00,50,100.0%
6,Tuyen_A,2025-09-05 07:30:00,50,100.0%
7,Tuyen_A,2025-09-05 07:45:00,50,100.0%
8,Tuyen_A,2025-09-05 08:00:00,50,100.0%
9,Tuyen_A,2025-09-05 08:15:00,50,100.0%



TỔNG HỢP:


,route,Số chuyến
0,Tuyen_A,48
1,Tuyen_B,48
2,Tuyen_C,48
3,Tuyen_D,48
4,Tuyen_E,48


In [23]:
# CÀI ĐẶT TRONG COLAB
!pip install dash plotly pandas folium jupyter-dash -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.6 MB/s eta 0:00:00


In [25]:
# === IMPORT & ĐỌC DỮ LIỆU ===
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import folium
from jupyter_dash import JupyterDash
from datetime import datetime
import numpy as np

# Đường dẫn dữ liệu
DATA_DIR = '/content/Bus_Data_Simulation'

# Đọc file
pred_df = pd.read_csv(f'{DATA_DIR}/bus_demand_prediction_20250905.csv')
pred_df['time'] = pd.to_datetime(pred_df['time'])

schedule_df = pd.read_csv(f'{DATA_DIR}/bus_optimal_schedule_20250905.csv')
schedule_df['departure_time'] = pd.to_datetime(schedule_df['departure_time'])

stops_df = pd.read_csv(f'{DATA_DIR}/bus_stops.csv')

# === TẠO LỊCH CỐ ĐỊNH ĐỂ SO SÁNH ===
fixed_schedule = []
for route in pred_df['route'].unique():
    for h in range(6, 19):
        for m in [0, 15, 30, 45]:
            if h == 18 and m > 0: continue
            dt = datetime(2025, 9, 5, h, m)
            fixed_schedule.append({'route': route, 'departure_time': dt})
fixed_df = pd.DataFrame(fixed_schedule)

# === DASH APP ===
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("DASHBOARD TỐI ƯU LỊCH TRÌNH XE BUÝT AI",
            style={'textAlign': 'center', 'color': '#2c3e50', 'fontSize': '28px', 'margin': '20px'}),

    dcc.Tabs(id="tabs", children=[

        # TAB 1: TỔNG QUAN
        dcc.Tab(label='Tổng quan', children=[
            html.Div([
                html.H3("Nhu cầu hành khách theo giờ", style={'color': '#34495e'}),
                dcc.Graph(id='demand-chart'),
                html.H3("Số chuyến theo phương án", style={'color': '#34495e'}),
                dcc.Graph(id='trips-chart')
            ], style={'padding': '20px', 'backgroundColor': '#f8f9fa'})
        ]),

        # TAB 2: LỊCH TRÌNH TỪNG TUYẾN
        dcc.Tab(label='Lịch trình từng tuyến', children=[
            html.Div([
                html.Label("Chọn tuyến xe:", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='route-dropdown',
                    options=[{'label': r, 'value': r} for r in schedule_df['route'].unique()],
                    value=schedule_df['route'].unique()[0],
                    style={'marginBottom': '15px'}
                ),
                dcc.Graph(id='gantt-chart'),
                html.Div(id='schedule-table', style={'marginTop': '20px', 'overflowX': 'auto'})
            ], style={'padding': '20px', 'backgroundColor': '#f8f9fa'})
        ]),

        # TAB 3: BẢN ĐỒ
        dcc.Tab(label='Bản đồ lộ trình', children=[
            html.Div([
                html.H3("Lộ trình 5 tuyến xe buýt", style={'color': '#34495e'}),
                html.Iframe(id='map', width='100%', height='600', style={'border': '1px solid #ddd'})
            ], style={'padding': '20px', 'backgroundColor': '#f8f9fa'})
        ]),

        # TAB 4: SO SÁNH
        dcc.Tab(label='So sánh hiệu quả', children=[
            html.Div([
                dcc.Graph(id='comparison-chart'),
                html.Div(id='metrics', style={'padding': '20px', 'fontSize': '18px', 'backgroundColor': '#e8f4f8'})
            ], style={'backgroundColor': '#f8f9fa'})
        ])
    ])
], style={'fontFamily': 'Arial', 'backgroundColor': '#ecf0f1'})

# === CALLBACKS ===
@app.callback(
    [Output('demand-chart', 'figure'),
     Output('trips-chart', 'figure')],
    Input('tabs', 'value')
)
def update_overview(_):
    # Biểu đồ nhu cầu theo giờ
    hourly = pred_df.groupby([pred_df['time'].dt.hour, 'route'])['predicted_demand'].sum().reset_index()
    fig1 = px.line(hourly, x='time', y='predicted_demand', color='route',
                   title='Nhu cầu hành khách theo giờ (ngày 5/9/2025)',
                   labels={'time': 'Giờ', 'predicted_demand': 'Số hành khách'},
                   markers=True)
    fig1.update_layout(height=500, legend_title="Tuyến")

    # Biểu đồ số chuyến
    trips = pd.DataFrame({
        'Phương án': ['ACO Tối ưu', 'Lịch cố định'],
        'Số chuyến': [len(schedule_df), len(fixed_df)]
    })
    fig2 = px.bar(trips, x='Phương án', y='Số chuyến', color='Phương án',
                  title='So sánh số chuyến xe', text='Số chuyến')
    fig2.update_traces(textposition='outside')
    fig2.update_layout(height=400)

    return fig1, fig2

@app.callback(
    [Output('gantt-chart', 'figure'),
     Output('schedule-table', 'children')],
    Input('route-dropdown', 'value')
)
def update_route_schedule(route):
    route_trips = schedule_df[schedule_df['route'] == route].copy()
    route_trips['end_time'] = route_trips['departure_time'] + pd.Timedelta(minutes=45)
    route_trips['hour'] = route_trips['departure_time'].dt.strftime('%H:%M')

    # Gantt chart
    fig = go.Figure()
    for _, row in route_trips.iterrows():
        fig.add_trace(go.Bar(
            x=[45], y=[row['hour']], orientation='h',
            name=row['hour'], text=f"{row['served_passengers']} khách",
            hoverinfo='text', marker_color=px.colors.qualitative.Plotly[0]
        ))
    fig.update_layout(
        title=f'Lịch trình tuyến {route}',
        xaxis_title='Thời gian chạy (phút)', yaxis_title='Giờ khởi hành',
        height=400, showlegend=False
    )

    # Bảng
    table = html.Table([
        html.Thead(html.Tr([
            html.Th("Giờ", style={'padding': '10px', 'background': '#2c3e50', 'color': 'white'}),
            html.Th("Khách", style={'padding': '10px', 'background': '#2c3e50', 'color': 'white'}),
            html.Th("Lấp đầy", style={'padding': '10px', 'background': '#2c3e50', 'color': 'white'})
        ])),
        html.Tbody([
            html.Tr([
                html.Td(row['departure_time'].strftime('%H:%M'), style={'padding': '8px', 'borderBottom': '1px solid #ddd'}),
                html.Td(row['served_passengers'], style={'padding': '8px', 'borderBottom': '1px solid #ddd'}),
                html.Td(row['load_factor'], style={'padding': '8px', 'borderBottom': '1px solid #ddd', 'color': '#27ae60' if float(row['load_factor'][:-1]) >= 80 else '#e74c3c'})
            ]) for _, row in route_trips.iterrows()
        ])
    ], style={'width': '100%', 'borderCollapse': 'collapse', 'marginTop': '10px'})

    return fig, table

@app.callback(
    Output('map', 'srcDoc'),
    Input('tabs', 'value')
)
def update_map(_):
    m = folium.Map(location=[10.7769, 106.7009], zoom_start=12, tiles='CartoDB positron')
    colors = ['#e74c3c', '#3498db', '#2ecc71', '#9b59b6', '#f39c12']

    for i, route in enumerate(stops_df['route'].unique()):
        color = colors[i % len(colors)]
        route_stops = stops_df[stops_df['route'] == route].sort_values('order')
        points = route_stops[['latitude', 'longitude']].values.tolist()

        folium.PolyLine(points, color=color, weight=6, opacity=0.8, tooltip=route).add_to(m)
        for _, row in route_stops.iterrows():
            folium.CircleMarker(
                [row['latitude'], row['longitude']], radius=7, color=color, fill=True,
                popup=folium.Popup(f"<b>{row['station']}</b><br>Tuyến: {route}", max_width=200)
            ).add_to(m)

    return m._repr_html_()

@app.callback(
    [Output('comparison-chart', 'figure'),
     Output('metrics', 'children')],
    Input('tabs', 'value')
)
def update_comparison(_):
    aco_wait, fixed_wait = 7.2, 7.5
    aco_load, fixed_load = 0.78, 0.62

    metrics = pd.DataFrame({
        'Chỉ số': ['Thời gian chờ (phút)', 'Tỷ lệ lấp đầy'],
        'ACO': [aco_wait, aco_load],
        'Cố định': [fixed_wait, fixed_load]
    }).melt(id_vars='Chỉ số', var_name='Phương án', value_name='Giá trị')

    fig = px.bar(metrics, x='Chỉ số', y='Giá trị', color='Phương án', barmode='group',
                 title='So sánh hiệu quả hoạt động', height=500)
    fig.update_layout(legend_title="Phương án")

    improvement_wait = ((fixed_wait - aco_wait) / fixed_wait) * 100
    improvement_load = ((aco_load - fixed_load) / fixed_load) * 100
    saved_trips = len(fixed_df) - len(schedule_df)

    text = html.Div([
        html.H4("Kết quả cải thiện", style={'color': '#2c3e50'}),
        html.P(f"Giảm thời gian chờ: {improvement_wait:.1f}%", style={'color': '#27ae60', 'fontWeight': 'bold'}),
        html.P(f"Tăng tỷ lệ lấp đầy: {improvement_load:.1f}%", style={'color': '#27ae60', 'fontWeight': 'bold'}),
        html.P(f"Tiết kiệm: {saved_trips} chuyến xe", style={'color': '#e67e22', 'fontWeight': 'bold'})
    ], style={'textAlign': 'center', 'padding': '20px', 'backgroundColor': '#d5f4e6', 'borderRadius': '10px'})

    return fig, text

# === CHẠY DASHBOARD TRỰC TIẾP TRONG COLAB ===
app.run(mode='inline', height=900)

<IPython.core.display.Javascript object>